# Getting Started with the Twingate API & Python

## Intro

The following Notebook runs through the fundamentals of using the Twingate API in Python.

## Prerequisites

Python3: If you can read this notebook, you should already have Python up and running ;)

You will need 2 additional things to make this notebook work:

* **an API Key** (with Read / Write permission)
* **the name of your Twingate tenant** (the * in *.twingate.com)

Once you have those 2 things, make sure to populate them in the next cell and you will be good to go!

In [38]:
# Change the API Token below to your own:
API_Token = '<My Twingate API Token>'

# Change the Twingate Network / Tenant name to your own:
Twingate_Network = '<My Twingate Network Name>'

# Once you have updated both fields, run this cell (by clicking on "Run" at the top of the page)
# (Running the page will effectively run the code within this cell and keep it all in-memory)
# You can ignore the next few lines: we are only importing libraries we will be using in this notebook

import requests
import json
import sys
import os

# Getting Started with Simple Calls

A GraphQL API is similar to a Rest API but offers a lot more control on what Queries you can send to the backend. The whole idea behind GraphQL is to be able to craft very targetted Queries in order to make it less compute-intensive for the backend to retrieve objects.

Let's explore a simple call we can use to retrieve a list of objects like resources, groups, users, etc.


In [39]:
# An API Call typically requires:
#  - a URL (where to send the API request)
#  - Headers (to set parameters like the API Authentication Token)
#  - a Body (with the specifics of the request itself)

# URL and Headers will mostly be the same for all API calls...
url = "https://"+Twingate_Network+".twingate.com/api/graphql/"

headers = {
  'X-API-KEY': API_Token,
  'Content-Type': 'application/json'
}

# However the Body is always different because it dictates what should be returned by the backend
QueryBody = """{
          devices(after: null, first:100) {
            edges {
              node {
                id
                name
              }
            }
          }
        }"""

# Let's take all of it together and send the API Call to the backend
response = requests.request("POST", url, headers=headers, json={'query': QueryBody})

# Let's print the response in a human-readable format
pretty_response = json.dumps(response.json(), indent=4)
print(pretty_response)

{
    "data": {
        "devices": {
            "edges": [
                {
                    "node": {
                        "id": "RGV2aWNlOjIwOTA1",
                        "name": "Brendon\u2019s MacBook Air"
                    }
                },
                {
                    "node": {
                        "id": "RGV2aWNlOjIxOTQ5",
                        "name": "PETER\u2019s MacBook Air"
                    }
                },
                {
                    "node": {
                        "id": "RGV2aWNlOjI4Njk0",
                        "name": "AF6528"
                    }
                },
                {
                    "node": {
                        "id": "RGV2aWNlOjI5ODg5",
                        "name": "MacBook Pro"
                    }
                },
                {
                    "node": {
                        "id": "RGV2aWNlOjI5ODk2",
                        "name": "Amy\u2019s MacBook Air"
                    }


### Anatomy of a Query

Let's take a look at the Body of the query itself and explain what all of it means:

{
    devices(after: null, first:100) {
        edges {
            node {
                id
                name
            }
        }
    }
}

* devices: the type of object we are retrieving information on.
* after: specifies what page to start the search from (more on Pagination below)
* first: specifies how many entries should be returned (in this case, the first 100 devices)
* edges & node: This is a GraphQL specific thing: edges and nodes is the typical structure data is returned as.
* id * name: Those are attributes of devices that we want the backend to return.



## Adding Attributes and Pagination

GraphQL allows you to add any attributes to your query as long as they are **valid attributes for the object type you are querying**.

For example, the "device" object type does have "id" and "name" as attributes but it also has others like:

* isTrusted
* osName
* deviceType

GraphQL also offers out of the box attributes for pagination (startCursor and hasNextPage, among others). Those are used to navigate large response sets that might contain more elements than what the "after" and "first" attributes allow for.

Let's rework our existing query and add a few of the things we just learned about:

In [51]:
# Let's get only the first device from the list of existing devices (first:1)
# let's also return the pagination info
# and add a few attributes to our Body so as to retrieve isTrusted, osName and deviceType for each Device:

QueryBody = """{
          devices(after: null, first:1) {
            edges {
              node {
                id
                name
                isTrusted
                osName
                deviceType
              }
            }
            pageInfo {
              startCursor
              hasNextPage
            }
          }
        }"""
    
response = requests.request("POST", url, headers=headers, json={'query': QueryBody})

pretty_response = json.dumps(response.json(), indent=4)
print(pretty_response)

# the following line allows us to parse the JSON response and store the content of 'startCursor' from it:
CursorID = response.json()['data']['devices']['pageInfo']['startCursor']



{
    "data": {
        "devices": {
            "edges": [
                {
                    "node": {
                        "id": "RGV2aWNlOjIwOTA1",
                        "name": "Brendon\u2019s MacBook Air",
                        "isTrusted": true,
                        "osName": "MAC_OS",
                        "deviceType": "LAPTOP"
                    }
                }
            ],
            "pageInfo": {
                "startCursor": "YXJyYXljb25uZWN0aW9uOjA=",
                "hasNextPage": true
            }
        }
    }
}


Great! We are now **getting a single object** in response but we are also retrieving more attributes.

The response also indicates that there are **more existing devices** ('hasNextPage' returns true). 

How would we extract the next 2 devices then?

We can do this by **leveraging Pagination** and the content of **'startCursor'** from the previous call and passing it in the **'after'** parameter of a new call.

Because the **CursorID is dynamically retrieved**, we will need to use a **variable and pass it to the body of the call**.



In [50]:
# Variables in a GraphQL Body should be defined in JSON format
# We are going to store the value of the cursor from the previous call into a variable called afterID:
variables = {"afterID":CursorID}

# Now, we need to change the body a bit to make room for the afterID variable:
QueryBody = """
    query ($afterID: String!){
        devices(after: $afterID, first:2) {
            edges {
                node {
                    id
                    name
                    isTrusted
                    osName
                    deviceType
               }
            }
            pageInfo {
              startCursor
              hasNextPage
            }
          }
        }"""

response = requests.request("POST", url, headers=headers, json={'query': QueryBody, 'variables': variables})

pretty_response = json.dumps(response.json(), indent=4)
print(pretty_response)

{
    "data": {
        "devices": {
            "edges": [
                {
                    "node": {
                        "id": "RGV2aWNlOjIxOTQ5",
                        "name": "PETER\u2019s MacBook Air",
                        "isTrusted": true,
                        "osName": "MAC_OS",
                        "deviceType": "LAPTOP"
                    }
                },
                {
                    "node": {
                        "id": "RGV2aWNlOjI4Njk0",
                        "name": "AF6528",
                        "isTrusted": true,
                        "osName": "MAC_OS",
                        "deviceType": "LAPTOP"
                    }
                }
            ],
            "pageInfo": {
                "startCursor": "YXJyYXljb25uZWN0aW9uOjE=",
                "hasNextPage": true
            }
        }
    }
}


Great! Now we get the next 2 devices and all attributes we wanted. 

What did we modify?

* we added at the beginning to specify that we are running a Query and that the Query will expect 1 variable called _"afterID"_ to be passed and that the parameter is expected to be a String
```python
query ($afterID: String!){
```

* we replaced the _"null"_ of the original query with our variable __$afterID__
    
* we added a field called __"variable"__ in our request and passed the value of $afterID in JSON format

## Getting Object Types and Available Fields

Now that we know how to work with Pagination, objects and fields, the question becomes two fold:

* what other objects can we work with?
* how do we determine all the valid fields for each object type?

In order to do both those things, we will use a GraphQL specific mechanism called **Introspection**. 

**Introspection** allows you to send "meta-queries" to the backend in order to get the backend to tell you what is available.

Think of it as a way for you to ask questions to the backend and expect responses that contain structural information but no actual data.

In [52]:
# The following Body can be used to retrieve all object types the backend supports:
QueryBody = """
{
  __schema {
    types {
      name
    }
  }
}
"""

response = requests.request("POST", url, headers=headers, json={'query': QueryBody})

pretty_response = json.dumps(response.json(), indent=4)
print(pretty_response)

{
    "data": {
        "__schema": {
            "types": [
                {
                    "name": "QueriesRoot"
                },
                {
                    "name": "ServiceAccountKey"
                },
                {
                    "name": "Node"
                },
                {
                    "name": "ID"
                },
                {
                    "name": "DateTime"
                },
                {
                    "name": "String"
                },
                {
                    "name": "ServiceAccountKeyStatus"
                },
                {
                    "name": "ServiceAccount"
                },
                {
                    "name": "ResourceConnection"
                },
                {
                    "name": "PageInfo"
                },
                {
                    "name": "Boolean"
                },
                {
                    "name": "ResourceEdge"
            

Looking at the response from the backend, it looks like it knows an object type called "ServiceAccount". 

Let's see if we can use that to **list the first 2 Service Accounts** in Twingate:


In [57]:
# Let's get only the first device from the list of existing devices (first:1)
# let's also return the pagination info
# and add a few attributes to our Body so as to retrieve isTrusted, osName and deviceType for each Device:

QueryBody = """{
          serviceAccounts(after: null, first:2) {
            edges {
              node {
                id
                name

              }
            }
            pageInfo {
              startCursor
              hasNextPage
            }
          }
        }"""
    
response = requests.request("POST", url, headers=headers, json={'query': QueryBody})

pretty_response = json.dumps(response.json(), indent=4)
print(pretty_response)




{
    "data": {
        "serviceAccounts": {
            "edges": [
                {
                    "node": {
                        "id": "U2VydmljZUFjY291bnQ6NzM5N2E2NTYtMDM1Ni00ZWU3LTlmMTgtYjFkNDdlMTY5MDI4",
                        "name": "CI/CD"
                    }
                }
            ],
            "pageInfo": {
                "startCursor": "YXJyYXljb25uZWN0aW9uOjA=",
                "hasNextPage": false
            }
        }
    }
}


It worked!

**Why did we use "serviceAccounts"** and not **"ServiceAccounts"** or **"ServiceAccount"** as specified by the backend in the previous response?

This is a quirck of GraphQL, since it returns lists, it expects the object type to be "pluralized" (from ServiceAccount to ServiceAccounts) and the standard convention is to use Camel Case when using those object types in a query... hence **serviceAccounts**.

### Other objects to try:

* Resources
* Remote Networks
* Connectors
* Users

In fact, let's try "Users" together.


In [58]:
# Let's get only the first device from the list of existing devices (first:1)
# let's also return the pagination info
# and add a few attributes to our Body so as to retrieve isTrusted, osName and deviceType for each Device:

QueryBody = """{
          users(after: null, first:2) {
            edges {
              node {
                id
                name

              }
            }
            pageInfo {
              startCursor
              hasNextPage
            }
          }
        }"""
    
response = requests.request("POST", url, headers=headers, json={'query': QueryBody})

pretty_response = json.dumps(response.json(), indent=4)
print(pretty_response)

{
    "errors": [
        {
            "message": "Cannot query field 'name' on type 'User'.",
            "locations": [
                {
                    "line": 6,
                    "column": 17
                }
            ],
            "path": null
        }
    ]
}


Oops, this didnt work. The error message seems to indicate that the object type **User** does not contain a field called **"name"**.

If **Users** don't have a **name** attribute then what attributed do they have?

Let's use introspection again, in a slightly different way this time:


In [59]:
# The following Body can be used to retrieve all object types the backend supports:
QueryBody = """
{
  __type(name: "User") {
    name
    kind
    fields {
        name
        type{
            name
            kind
        }
    }
  }
}
"""

response = requests.request("POST", url, headers=headers, json={'query': QueryBody})

pretty_response = json.dumps(response.json(), indent=4)
print(pretty_response)

{
    "data": {
        "__type": {
            "name": "User",
            "kind": "OBJECT",
            "fields": [
                {
                    "name": "id",
                    "type": {
                        "name": null,
                        "kind": "NON_NULL"
                    }
                },
                {
                    "name": "createdAt",
                    "type": {
                        "name": null,
                        "kind": "NON_NULL"
                    }
                },
                {
                    "name": "updatedAt",
                    "type": {
                        "name": null,
                        "kind": "NON_NULL"
                    }
                },
                {
                    "name": "firstName",
                    "type": {
                        "name": null,
                        "kind": "NON_NULL"
                    }
                },
                {
                    "name":

Great! By using the previous query with the Object Type **User**, we get a full list of attributes for Users and, unsurpisingly, **name** is not present; however we now know that other attributes can be used like:

* firstName
* lastName
* isAdmin
* etc..

Let's go back to our previous call with some of those new attributes:



In [60]:
# Let's get only the first device from the list of existing devices (first:1)
# let's also return the pagination info
# and add a few attributes to our Body so as to retrieve isTrusted, osName and deviceType for each Device:

QueryBody = """{
          users(after: null, first:1) {
            edges {
              node {
                id
                firstName
                lastName
                isAdmin

              }
            }
          }
        }"""
    
response = requests.request("POST", url, headers=headers, json={'query': QueryBody})

pretty_response = json.dumps(response.json(), indent=4)
print(pretty_response)




{
    "data": {
        "users": {
            "edges": [
                {
                    "node": {
                        "id": "VXNlcjoxOTU5NA==",
                        "firstName": "Alex",
                        "lastName": "Marshall",
                        "isAdmin": true
                    }
                }
            ]
        }
    }
}


### Excellent! We now know:

* How to run a query to extract a list of objects (from multiple pages)
* How to determine the list of Object Types known by the Twingate backend
* How to determine the list of Attributes available for each Object Type
* How to use variables in a GraphQL Query

# Updating Objects / Working with Mutations

Retrieving data from the twingate backend is useful however being able to create, delete or update existing objects is equally important.

In the GraphQL world, all operations that deal with modifications are called **Mutations**. 

Let's explore Mutations together.

## Mutations

Think of Mutations as similar to Functions in any other language. A Mutation is designed to carry out a particular operation and is usually named after what it does. Here are some examples of Mutations made available by the Twingate API:

* deviceUpdate
* resourceUpdate
* resourceCreate
* etc.

Like Functions, Mutations take various input parameters and produce an output.

Let's start with simple Mutations.


### Mark a Device as Trusted (or Untrusted)

The Mutation available to update a device is called **updateDeviceTrust**, as you would expect, if we are going to update a device's Trusted attribute, we will need to provide:

* the unique ID of the Device we want to update
* a value set to true or false depending on whether we want the device to be trusted or not

You might have guessed it already, those 2 pieces of information are going to be the input parameters to the **updateDeviceTrust** Mutation.

!! **You will need to replace the id below with your own device ID before running the cell**

In [61]:
# the device ID below should be replaced by your own device ID

QueryBody = """
    mutation{
        deviceUpdate(id: "RGV2aWNlOjE4NzIyMg==", isTrusted: true) {
          ok
          error
        }
    }
"""
    
response = requests.request("POST", url, headers=headers, json={'query': QueryBody})

pretty_response = json.dumps(response.json(), indent=4)
print(pretty_response)

{
    "data": {
        "deviceUpdate": {
            "ok": true,
            "error": null
        }
    }
}


Provided the ID is correct, you should get the following response:

```javascript
{
    "data": {
        "deviceUpdate": {
            "ok": true,
            "error": null
        }
    }
}
```

Not too much to see in the response except for 2 things:

* "ok" returns true: this is a default output of GraphQL mutations. It means the backend carried out the operation successfully
* "error" return null: this is also a defaut output. it means no error was returned by the backend

Great! this means our call worked as expected. If you go check the Admin Console, your device should now be marked as Trusted.



### Adding Variables & adding output information

By now, you know that GraphQL is a query language and expects the Client to specify what should be returned. 

In the previous example, we are only asking the backend to return the "ok" and "error" flags with nothing else but **what if we wanted the backend to return the new isTrusted value** after the update?

Luckily, since this is a design principle of GraphQL, we can (and while we are at it, let's variabilize the device ID as well).



In [63]:
# Modify the myID value to your own Device ID
myID = "RGV2aWNlOjE4NzIyMg=="

# Modify True to False if you wish to untrust a device as opposed to trust it
variables = {"deviceID":myID ,"isTrusted":True}

QueryBody = """
               mutation 
                   updateDeviceTrust($deviceID: ID!, $isTrusted: Boolean!){
                        deviceUpdate(id: $deviceID, isTrusted: $isTrusted) {
                              ok
                              error
                              entity {
                                id
                                name
                                isTrusted
                              }
                        }
                 }
            """

response = requests.request("POST", url, headers=headers, json={'query': QueryBody, 'variables': variables})

pretty_response = json.dumps(response.json(), indent=4)
print(pretty_response)


{
    "data": {
        "deviceUpdate": {
            "ok": true,
            "error": null,
            "entity": {
                "id": "RGV2aWNlOjE4NzIyMg==",
                "name": "brendan\u2019s MacBook Air",
                "isTrusted": true
            }
        }
    }
}


If you used a valid Device ID, you should see a response similar to this:

```javascript
{
    "data": {
        "deviceUpdate": {
            "ok": true,
            "error": null,
            "entity": {
                "id": "RGV2aWNlOjE4NzIyMg==",
                "name": "brendan\u2019s MacBook Air",
                "isTrusted": true
            }
        }
    }
}
```

Great! Now or update request returns the updated status of our device.


### Finding Other Mutations

Let's explore more Mutations in the following section, now that we have the fundamentals of Queries and Mutations down.

Because it is GraphQL, you probably understand by now that there must be a meta-query we can send to the backend in order to retrieve the list of **all available mutations** .. and you are right.

Let's run it!

In [64]:
QueryBody = """
               query {
  __schema {
    mutationType {
      name
      fields {
        name
        args {
          name
          defaultValue
          type {
            ...TypeRef
          }
        }
      }
    }
  }
}

fragment TypeRef on __Type {
  kind
  name
  ofType {
    kind
    name
    ofType {
      kind
      name
      ofType {
        kind
        name
        ofType {
          kind
          name
          ofType {
            kind
            name
            ofType {
              kind
              name
              ofType {
                kind
                name
              }
            }
          }
        }
      }
    }
  }
}
"""

response = requests.request("POST", url, headers=headers, json={'query': QueryBody})

pretty_response = json.dumps(response.json(), indent=4)
print(pretty_response)


{
    "data": {
        "__schema": {
            "mutationType": {
                "name": "MutationsRoot",
                "fields": [
                    {
                        "name": "serviceAccountKeyCreate",
                        "args": [
                            {
                                "name": "expirationTime",
                                "defaultValue": null,
                                "type": {
                                    "kind": "NON_NULL",
                                    "name": null,
                                    "ofType": {
                                        "kind": "SCALAR",
                                        "name": "Int",
                                        "ofType": null
                                    }
                                }
                            },
                            {
                                "name": "name",
                                "defaultValue": "null",
     

### Finding Mutations and their Arguments

If you browse the JSON response above, you will notice a list of **fields** under **"mutationType"**. 

Each field has:

* a "name" attribute: this is the name to use to use the mutation
* a list of "args": each arg can be seen as a potential input parameter for the mutation

Let's examine a mutation we have already used: **deviceUpdate**

```javascript
{
                        "name": "deviceUpdate",
                        "args": [
                            {
                                "name": "id",
                                "defaultValue": null,
                                "type": {
                                    "kind": "NON_NULL",
                                    "name": null,
                                    "ofType": {
                                        "kind": "SCALAR",
                                        "name": "ID",
                                        "ofType": null
                                    }
                                }
                            },
                            {
                                "name": "isTrusted",
                                "defaultValue": "null",
                                "type": {
                                    "kind": "SCALAR",
                                    "name": "Boolean",
                                    "ofType": null
                                }
                            }
                        ]
                    }
```

From this definition, we can quickly see that **deviceUpdate** provides only 2 possible input parameters (id and isTrusted).

You will also notice that:

* the "id" attribute has a Type Name of "ID"
* the "isTrusted" attribute has a Type Name of "Boolean"

This should help you explain why the signature to our mutation contained **ID!** and **Boolean!**

```python
updateDeviceTrust($deviceID: ID!, $isTrusted: Boolean!){}
```


### Using Other Mutations

Let's explore 1 more Mutation before we wrap up. Let's figure out how to **add a user to an existing group**.

Let's first figure out what Mutation to use and what its arguments might be:

```javascript
{
                        "name": "groupUpdate",
                        "args": [
                            {
                                "name": "addedResourceIds",
                                "defaultValue": "null",
                                "type": {
                                    "kind": "LIST",
                                    "name": null,
                                    "ofType": {
                                        "kind": "SCALAR",
                                        "name": "ID",
                                        "ofType": null
                                    }
                                }
                            },
                            {
                                "name": "addedUserIds",
                                "defaultValue": "null",
                                "type": {
                                    "kind": "LIST",
                                    "name": null,
                                    "ofType": {
                                        "kind": "SCALAR",
                                        "name": "ID",
                                        "ofType": null
                                    }
                                }
                            },
                            {
                                "name": "id",
                                "defaultValue": null,
                                "type": {
                                    "kind": "NON_NULL",
                                    "name": null,
                                    "ofType": {
                                        "kind": "SCALAR",
                                        "name": "ID",
                                        "ofType": null
                                    }
                                }
                            },
                            {
                                "name": "isActive",
                                "defaultValue": "null",
                                "type": {
                                    "kind": "SCALAR",
                                    "name": "Boolean",
                                    "ofType": null
                                }
                            },
                            {
                                "name": "name",
                                "defaultValue": "null",
                                "type": {
                                    "kind": "SCALAR",
                                    "name": "String",
                                    "ofType": null
                                }
                            },
                            {
                                "name": "removedResourceIds",
                                "defaultValue": "null",
                                "type": {
                                    "kind": "LIST",
                                    "name": null,
                                    "ofType": {
                                        "kind": "SCALAR",
                                        "name": "ID",
                                        "ofType": null
                                    }
                                }
                            },
                            {
                                "name": "removedUserIds",
                                "defaultValue": "null",
                                "type": {
                                    "kind": "LIST",
                                    "name": null,
                                    "ofType": {
                                        "kind": "SCALAR",
                                        "name": "ID",
                                        "ofType": null
                                    }
                                }
                            },
                            {
                                "name": "resourceIds",
                                "defaultValue": "null",
                                "type": {
                                    "kind": "LIST",
                                    "name": null,
                                    "ofType": {
                                        "kind": "SCALAR",
                                        "name": "ID",
                                        "ofType": null
                                    }
                                }
                            },
                            {
                                "name": "userIds",
                                "defaultValue": "null",
                                "type": {
                                    "kind": "LIST",
                                    "name": null,
                                    "ofType": {
                                        "kind": "SCALAR",
                                        "name": "ID",
                                        "ofType": null
                                    }
                                }
                            }
                        ]
                    }
```

There is a lot more in there than for deviceUpdate. The good news is that we will not need all the arguments in the response: those are the **possible** arguments you can pass but not all of them are required depending on what you want to achieve.

If we want to add a user to a group, it makes sense that we would need:
* the unique Group ID to update (the argument for this is **"id"**)
* the unique User IS to add to the Group (the argument for this is **addedUserIds** which seems to be a list of IDs as opposed to a single one, more on that later).




In [65]:
# Before running this cell, update the following 2 IDs to add your own User to a valid Group
myGroupID = "R3JvdXA6NjQxMA=="
myUserID = "VXNlcjoxMTMyNTY="

variables = {"GroupID":myGroupID ,"userIds":[myUserID]}

QueryBody = """
               mutation 
                   addUserToGroup($GroupID: ID!, $userIds:[ID]){

                        groupUpdate(id: $GroupID, addedUserIds:$userIds) {
                              ok
                              error
                              entity {
                                id
                                name
                              }
                        }
                 }
            """

response = requests.request("POST", url, headers=headers, json={'query': QueryBody, 'variables': variables})

pretty_response = json.dumps(response.json(), indent=4)
print(pretty_response)


{
    "data": {
        "groupUpdate": {
            "ok": true,
            "error": null,
            "entity": {
                "id": "R3JvdXA6NjQxMA==",
                "name": "IT Admins"
            }
        }
    }
}


If you used a valid User ID and Group ID, you should get a response that looks like the following:

```javascript
{
    "data": {
        "groupUpdate": {
            "ok": true,
            "error": null,
            "entity": {
                "id": "R3JvdXA6NjQxMA==",
                "name": "IT Admins"
            }
        }
    }
}
```

Great! If you check the Admin Console, you will see that your User is now part of the Group.

### One Last Thing - Nested Output

What if you also wanted to retrieve complex information on the Group you are updating as you update it? We have already seen that you can craft elaborate queries in GraphQL so we should be able to ask the backend to also return in its output:

* the list of users from the group
* the list of resources attached to the group

Let's take a look at the updated version of our groupUpdate mutation:

In [70]:
# Before running this cell, update the following 2 IDs to add your own User to a valid Group
myGroupID = "R3JvdXA6NjQxMA=="
myUserID = "VXNlcjoxMTMyNTY="

variables = {"GroupID":myGroupID ,"userIds":[myUserID]}

QueryBody = """
               mutation 
                   addUserToGroup($GroupID: ID!, $userIds:[ID]){

                        groupUpdate(id: $GroupID, addedUserIds:$userIds) {
                              ok
                              error
                              entity {
                                id
                                name
                                users 
                                {
                                    edges{
                                        node{
                                             id
                                             email
                                             firstName
                                             lastName
                                            }
                                        }
                                }
                                resources 
                                {
                                    edges{
                                        node{
                                              id
                                              name
                                              address {
                                                  type
                                                  value
                                              }
                                              isActive
                                          }
                                }
                                }
                              }
                        }
                 }
            """

response = requests.request("POST", url, headers=headers, json={'query': QueryBody, 'variables': variables})

pretty_response = json.dumps(response.json(), indent=4)
print(pretty_response)


{
    "data": {
        "groupUpdate": {
            "ok": true,
            "error": null,
            "entity": {
                "id": "R3JvdXA6NjQxMA==",
                "name": "IT Admins",
                "users": {
                    "edges": [
                        {
                            "node": {
                                "id": "VXNlcjoxOTU5NA==",
                                "email": "alexm@twindemo.com",
                                "firstName": "Alex",
                                "lastName": "Marshall"
                            }
                        },
                        {
                            "node": {
                                "id": "VXNlcjoxOTU5OA==",
                                "email": "tony@twingate.com",
                                "firstName": "Tony",
                                "lastName": "Huie"
                            }
                        },
                        {
                          

Great! You now also see the full list of Users and Resources in the output of your API call, even though those lists are nested.